In [5]:
# Base Libraries
import numpy as np
import pandas as pd
import datetime
import warnings

warnings.filterwarnings('ignore')

In [6]:
# Read files to dataframes
ais_all_data = pd.read_csv("Maersk_AIS.csv", header = 0, parse_dates = ['timestamp'])
container_data = pd.read_csv("Maersk_Container_Ships.csv")
port_locations = pd.read_csv("ports_location.csv")

In [7]:
# Remove noises and redundant data
ais_data = ais_all_data.query('naviation_status in["At Anchor", "Moored", "Under Way", "Under way sailing"]')
ais_data = ais_data.sort_values(by=['imo','timestamp'])
ais_data.drop_duplicates(['imo','timestamp'], keep = 'first', inplace = True)

In [8]:
# Normalize the AIS data
ais_data['prev_dt'] = ais_data.groupby(['imo'])['timestamp'].shift(1)
ais_data['date_diff'] = ais_data['timestamp'] - ais_data['prev_dt']
ais_data['date_diff_secs'] = np.where((ais_data['date_diff'].dt.total_seconds() == np.nan), np.nan_to_num(0), ais_data['date_diff'].dt.total_seconds())
ais_data['prev_sts'] = ais_data.groupby(['imo'])['naviation_status'].shift(1)

# Select records when the time difference is 1hr
ais_data['tripid'] = ((ais_data['naviation_status'] != ais_data['prev_sts']) | (ais_data['date_diff_secs'] > 3600)).groupby(ais_data['imo']).cumsum()

ais_first = ais_data.groupby(['imo','tripid'],as_index = False)['timestamp'].min()
ais_compressed = ais_first.merge(ais_data, how = 'inner', on=['imo', 'tripid', 'timestamp'])

# Merge the filtered data with Maersk dataframe
maersk_data = ais_compressed.merge(container_data[['imo', 'mmsi','name','teuCapacity']], how = 'left', on = 'imo')

In [ ]:
# Determine Origin-Destination / Port / Terminal
maersk_ports = maersk_data.query('naviation_status in["At Anchor", "Moored", "Under Way"] and speed < 3 and distance_from_nearest_port < 30')
maersk_ports = maersk_ports.sort_values(by=['imo','timestamp'])
maersk_ports['nearest_port_name'] = maersk_ports['nearest_port_name'].str.replace(' Anch.','')
maersk_ports['nearest_port_name'] = maersk_ports['nearest_port_name'].str.replace('Singapore LNG Terminal','Singapore')
maersk_ports = maersk_ports.rename({'nearest_port_name': 'port'}, axis='columns')

maersk_ports = (maersk_ports.merge(port_locations, on='port', how='left', suffixes=('_','')))
maersk_ports = maersk_ports.sort_values(by=['imo','timestamp'])
maersk_ports['port_to'] = maersk_ports['port'].shift(-1)
maersk_ports['lng_to'] = maersk_ports['lng'].shift(-1)
maersk_ports['lat_to'] = maersk_ports['lat'].shift(-1)
possible_ports = maersk_ports.copy()
maersk_ports = maersk_ports.query('port != port_to')
maersk_ports.to_csv('Maersk_Ports.csv')

In [ ]:
# Prepare final dataset with Ports and Voyages
maersk_final = maersk_data.merge(maersk_ports, how = 'left')
maersk_final = maersk_final.drop(columns = ['id', 'nearest_port_id', 'eta', 'source_type', 'prev_dt', 'date_diff', 'prev_sts', 'area', 'type', 'country_code', 'country_name'])
maersk_final['latitude'] = np.where((pd.notnull(maersk_final['port'])), maersk_final['lat'], maersk_final['latitude'])
maersk_final['longitude'] = np.where((pd.notnull(maersk_final['port'])), maersk_final['lng'], maersk_final['longitude'])
maersk_final = maersk_final.drop(columns = ['nearest_port_name', 'distance_from_nearest_port', 'destination', 'heading', 'speed', 'course', 'draught', 'date_diff_secs', 'mmsi', 'name', 'teuCapacity'])
maersk_final.to_csv('Maersk_Final.csv')

In [ ]:
# Melt Maersk Ports data to find Voyages
maersk_temp = maersk_ports.drop(columns = ['id', 'nearest_port_id', 'source_type', 'prev_dt', 'date_diff', 'prev_sts', 'area', 'type', 'country_code', 'country_name', 'distance_from_nearest_port', 'course', 'draught', 'heading', 'mmsi', 'latitude', 'longitude', 'tripid', 'port_to', 'lat_to', 'lng_to'])

maersk_melted = (maersk_temp.assign(idx=maersk_temp.groupby('imo').cumcount())
   .pivot(index='idx',columns='imo', values=maersk_temp.columns)
   .swaplevel(0,1, axis=1)
   .sort_index(axis=1)
)
maersk_melted.to_csv('Maersk_Melted.csv')

In [ ]:
# Find Services from Maersk Ports data
maersk_service = maersk_temp.drop(columns = ['timestamp', 'naviation_status', 'date_diff_secs', 'name', 'lng', 'lat', 'teuCapacity'])
maersk_service = (maersk_service.groupby(['imo'])
      .agg({'port': lambda x: x.tolist()})
      .reset_index())

maersk_service.to_csv('Maersk_Services.csv')